### Libs

In [1]:
import openpyxl
from openpyxl import load_workbook
import pandas as pd
from PyQt5.QtWidgets import QApplication, QFileDialog
import sys
import os
import re
import datetime as dt

### Get folder and mapacaixa files

In [2]:
app = QApplication(sys.argv)
folder = QFileDialog.getExistingDirectory(None, "Select Folder")

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: não é possível abrir arquivo compartilhado: Arquivo ou diretório inexistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: não é possível abrir arquivo compartilhado: Arquivo ou diretório inexistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: não é possível abrir arquivo compartilhado: Arquivo ou diretório inexistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [3]:
files = os.listdir(folder)

In [4]:
files

['Mapa Caixa noite 01-11-24.xlsx', 'Mapa Caixa 01-11-24.xlsx']

In [5]:
sheetnames = [f for f in files if re.match(r'[mM].+?\.xlsx$', f)]
night_sheetnames =  [sheet for sheet in sheetnames if re.search("[nN][oO][iI][tT][eE]", sheet)]
day_sheetnames = [sheet for sheet in sheetnames if sheet not in night_sheetnames]

In [6]:
day_sheetnames, night_sheetnames

(['Mapa Caixa 01-11-24.xlsx'], ['Mapa Caixa noite 01-11-24.xlsx'])

In [7]:
start_copy_cell = "AE7"
end_copy_cell = "AE60"

### Create new LC

In [8]:
month = 7
year = 2025
first_dt_cell = "C4"
first_dt_weekname_cell = "C3"
new_lc_namefile = f"LC_{month}_{year}.xlsx"

### Open LC Sheets

In [9]:
base_lc_filename = QFileDialog.getOpenFileName(None, "Select Base LC File", filter="Excel Files (*.xlsx)")[0]
new_lc = load_workbook(base_lc_filename, keep_vba=True)
lc_sheetnames = new_lc.sheetnames
lc_day_sheetname = [sheet for sheet in lc_sheetnames if "dia" in sheet.lower()][0]
lc_day_sheet = new_lc[lc_day_sheetname]
lc_night_sheetname = [sheet for sheet in lc_sheetnames if "noite" in sheet.lower()][0]
lc_night_sheet = new_lc[lc_night_sheetname]

### Set the dates correctly

In [10]:
first_dt = dt.datetime(day=1, month=month, year=year)
first_dt_weekname = first_dt.strftime("%A")[0:3]
lc_day_sheet[first_dt_cell] = first_dt
lc_day_sheet[first_dt_weekname_cell] = first_dt_weekname

### Copy and paste values from MapaCaixa to LC

In [11]:
def copy_and_paste_values(lc_sheet, sheetnames, start_copy_cell, end_copy_cell):
    for sheetname in sheetnames:
        filename = os.path.join(folder, sheetname)
        workbook = load_workbook(filename, keep_vba=True, data_only=True)
        active_sheet = workbook[workbook.sheetnames[0]]
        # Extract values from the range
        values = [cell[0].value for cell in active_sheet[start_copy_cell:end_copy_cell]]
        sheet_dt = re.findall(r"\d+", sheetname)
        sheet_day = int(sheet_dt[0])
        #sheet_dt = dt.datetime(day=sheet_dt[0], month=sheet_dt[1], year=sheet_dt[2])
        paste_column = openpyxl.utils.get_column_letter(2 + sheet_day) #Starts on C and C is the 3rd column
        start_paste_cell = f"{paste_column}8"
        end_paste_cell = f"{paste_column}61"
        print(start_paste_cell, end_paste_cell)
        print(values)
        # Copy values to the new sheet
        cells = lc_sheet[start_paste_cell:end_paste_cell]
        for i, cell in enumerate(cells):
            cell[0].value = values[i]  
        
        

In [12]:
copy_and_paste_values(lc_day_sheet, day_sheetnames, start_copy_cell, end_copy_cell)

C8 C61
[datetime.datetime(2024, 11, 1, 0, 0), 0, 8274.64, 144, 0, 0, 0, 468.5, 343.65, 316.69, 2794.41, 4047.8, 0, 0, 0, 304.77, -1.1799999999998363, None, None, None, 3846.72, 73, 49.9, 1, 0, 0, 0, 0, 3846.72, 0, 42.9, 0, 0, 49.9, 0, 0, 0, 0, 0, None, 0, 42.90000000000009, 0, 0, 576.5, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [13]:
copy_and_paste_values(lc_night_sheet, night_sheetnames, start_copy_cell, end_copy_cell)

C8 C61
[datetime.datetime(2024, 11, 1, 0, 0), 0, 6334.6, 83, 0, 0, 0, 220, 137.17, 150, 700.91, 4679.32, 0, 0, 447.20000000000005, 0, None, 6.821210263296962e-13, None, None, 278.6, 5, 104.9, 1, 0, 0, 0, 0, 276.1, 0, 0, 0, 0, 104.9, 0, 0, 0, 0, 0, 0, 2.5, 0, 2, 3, 4, 0, 68, 4, 84, 0, None, None, None, None]


### Save new LC

In [14]:

new_lc.save(new_lc_namefile)

# Testes

In [43]:
start_copy_cell, end_copy_cell

('AE7', 'AE60')

In [30]:
filename = os.path.join(folder, day_sheetnames[0])
workbook = load_workbook(filename, keep_vba=True, data_only=True)
active_sheet = workbook[workbook.sheetnames[0]]
# Extract values from the range
values = [cell[0].value for cell in active_sheet[start_copy_cell:end_copy_cell]]

In [31]:
active_sheet[start_copy_cell:end_copy_cell][1][0].value

0

In [32]:
print(filename)
print(start_copy_cell, end_copy_cell)
print(values)

/home/victor/Documentos/mapacaixapier/novembro/Mapa Caixa 01-11-24.xlsx
AE7 AE60
[datetime.datetime(2024, 11, 1, 0, 0), 0, 8274.64, 144, 0, 0, 0, 468.5, 343.65, 316.69, 2794.41, 4047.8, 0, 0, 0, 304.77, -1.1799999999998363, None, None, None, 3846.72, 73, 49.9, 1, 0, 0, 0, 0, 3846.72, 0, 42.9, 0, 0, 49.9, 0, 0, 0, 0, 0, None, 0, 42.90000000000009, 0, 0, 576.5, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [59]:
active_sheet["AE7"].value

datetime.datetime(2024, 11, 2, 0, 0)